IMPORTING ESSENTIAL LIBRARIES
TENSORFLOW FOR TRAINIG (Layers,Sequntial,Keras)
PATHLIB FOR DATASET FETCHING (Path)
NUMPY FOR DATA PROCESSING (Numpy Array {.array} conversion)
OPENCV (cv2) FOR IMAGE PROCESSING (imread)
MATHPLOTLIB.PYPLOT FOR IMAGE VIEWING (imshow)

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers , Sequential
import numpy as np
#import pandas as pd
import cv2
import matplotlib.pyplot as plt
import pathlib
%matplotlib inline

Pointer to Train, Test, Validation File Directories

In [7]:
dataset_dir_train = pathlib.Path("Potato_Dataset/Train") #Train Directory
dataset_dir_test = pathlib.Path("Potato_Dataset/Test") #Test Directory
dataset_dir_valid = pathlib.Path("Potato_Dataset/Valid") #Valid Directory
dataset_dir_test

PosixPath('Potato_Dataset/Test')

Labels for Classification, Currently 3.
Datastructure Dictionary Used.

In [8]:
dataset_label = {
    "Potato Early Blight" : 0,
    "Potato Late Blight" : 1,
    "Healthy" : 2
}

File Paths in Each Directory Converted grouped into list each having their Classification Name as Key. 
DataStructure Dictionary and List are used. 

Train Folder has 3 Directories for 3 different Classes.
Similarly Test and Valid Dictionaries Created.

In [9]:
dataset_dictionary_train = {

    "Potato Early Blight" : list(dataset_dir_train.glob("Potato___Early_blight/*")),
    "Potato Late Blight" : list(dataset_dir_train.glob("Potato___Late_blight/*")),
    "Healthy" : list(dataset_dir_train.glob("Potato___healthy/*"))
    
}

dataset_dictionary_train["Potato Early Blight"]

In [5]:
dataset_dictionary_valid = {

    "Potato Early Blight" : list(dataset_dir_valid.glob("Potato___Early_blight/*")),
    "Potato Late Blight" : list(dataset_dir_valid.glob("Potato___Late_blight/*")),
    "Healthy" : list(dataset_dir_valid.glob("Potato___healthy/*"))
    
}

In [6]:
dataset_dictionary_test = {

    "Potato Early Blight" : list(dataset_dir_test.glob("Potato___Early_blight/*")),
    "Potato Late Blight" : list(dataset_dir_test.glob("Potato___Late_blight/*")),
    "Healthy" : list(dataset_dir_test.glob("Potato___healthy/*"))
    
}

Image Files are Read using File Paths and append to a X_list and it's corresponding class label into y_list.

Here label defines each key in Dictionary and images specify each value of that key in essence here it would be the list of File Paths.
In Inner loop image is each element in images (List of File Paths)
img will be an array(3D) returned from imread function using Corresponding File Paths.
Image Then appended to X_list
The integer value of key of image (it's label) is append to y_list.

Aim :  TO CREATE X_test,X_traiin,X_valid,y_train,y_test,y_valid lists

In [7]:
X_train = []
y_train = []
X_train_names = []

for label, images in dataset_dictionary_train.items():
    for image in images:
        img = cv2.imread(str(image))
        X_train.append(img)
        X_train_names.append(str(image))
        y_train.append(dataset_label[label])
        

X_train[0]

In [8]:
X_test = []
y_test = []
X_test_names = []

for label, images in dataset_dictionary_test.items():
    for image in images:
        img = cv2.imread(str(image))
        X_test.append(img)
        X_test_names.append(str(image))
        y_test.append(dataset_label[label])
        

In [9]:
X_valid = []
y_valid = []
X_valid_names = []

for label, images in dataset_dictionary_valid.items():
    for image in images:
        img = cv2.imread(str(image))
        X_valid.append(img)
        X_valid_names.append(str(image))
        y_valid.append(dataset_label[label])
        

1 st Layer of Model
It Resize and Rescales the Images

In [10]:
resize_and_rescale = tf.keras.Sequential([

    tf.keras.layers.experimental.preprocessing.Resizing(256,256),
    tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255)

])

2nd Layer of Model for Data Augmentation

In [11]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
])

Sequential Model Specified 
Having Layers
1. Rescale
2. Data Augmentation
3. 4 Convolutional Layer
4. 4 Pooling Layers
5. Flatten Layer
6. 2 Dense Layer with first having 20% Dropout
7. Output Layer (3 nodes) 

In [12]:
model1 = Sequential([

    resize_and_rescale,
    data_augmentation,

    layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=(256,256,3),padding='same'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'),
    layers.MaxPooling2D(pool_size=(2,2)),

    layers.Flatten(),

    layers.Dense(128,activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64,activation='relu'),

    layers.Dense(3,activation='softmax'),

])

Building Model for Summary Analysis

In [13]:
model1.build((None,256,256,3))

In [14]:
model1.summary() #Model Summary

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 sequential_1 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 64, 64, 64)        0         
 g2D)                                                 

Converting X_list and y_list to Numpy Array for Model Training.

In [15]:
X_train_np = np.array(X_train)
y_train_np = np.array(y_train)

In [16]:
X_valid_np = np.array(X_valid)
y_valid_np = np.array(y_valid)

Compile Model

In [17]:
model1.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Training Model

In [18]:
history = model1.fit(X_train_np,y_train_np,epochs=100, validation_data=(X_valid_np,y_valid_np))

Epoch 1/100
29/29 [==============================] - 40s 1s/step - loss: 1.1027 - accuracy: 0.3722 - val_loss: 0.9683 - val_accuracy: 0.5833
Epoch 2/100
29/29 [==============================] - 40s 1s/step - loss: 0.6817 - accuracy: 0.6856 - val_loss: 0.5335 - val_accuracy: 0.7667
Epoch 3/100
29/29 [==============================] - 40s 1s/step - loss: 0.4024 - accuracy: 0.8322 - val_loss: 0.5869 - val_accuracy: 0.7433
Epoch 4/100
29/29 [==============================] - 39s 1s/step - loss: 0.3607 - accuracy: 0.8633 - val_loss: 0.4110 - val_accuracy: 0.8233
Epoch 5/100
29/29 [==============================] - 39s 1s/step - loss: 0.3465 - accuracy: 0.8611 - val_loss: 0.3533 - val_accuracy: 0.8367
Epoch 6/100
29/29 [==============================] - 39s 1s/step - loss: 0.3190 - accuracy: 0.8700 - val_loss: 0.2857 - val_accuracy: 0.8667
Epoch 7/100
29/29 [==============================] - 39s 1s/step - loss: 0.3129 - accuracy: 0.8678 - val_loss: 0.2744 - val_accuracy: 0.8667
Epoch 8/100
2

Convert X_test and y_test to Numpy Array for Model Evaluation

In [22]:
X_test_np = np.array(X_test)
y_test_np = np.array(y_test)

Model Evaluation

In [23]:
model1.evaluate(X_test_np,y_test_np)

10/10 [==============================] - 2s 240ms/step - loss: 0.0881 - accuracy: 0.9767


[0.08814892172813416, 0.9766666889190674]

Model Saving as Folder

In [24]:
model1.save(f"Models/{2}")

INFO:tensorflow:Assets written to: Models/2/assets


INFO:tensorflow:Assets written to: Models/2/assets
